In [1]:
import sys
sys.path.append('/home/trukhinmaksim/src')
import json

In [2]:
from time import time
from src.utils.CacheAdapter import FlatAdapter, EXP_END_OF_DATA, createAdapter_02_04_25_GOOD, createTrainSetAdapter_02_04_25_GOOD, createTestSetAdapter_02_04_25_GOOD
from src.utils.Corpus import CacheCorpus, FlatCorpus
from src.utils.DatasetManager import ProjectsDatasetManager
from src.Doc2Vec_model import Model


from gensim.models.doc2vec import TaggedDocument


In [3]:
adapter = createAdapter_02_04_25_GOOD()#JSONMultiFileAdapter(CACHE_FILE_NAME)
manager = ProjectsDatasetManager(50, cacheAdapter = adapter)
cursor = CacheCorpus(manager, limit = 10)

flatAda = createTrainSetAdapter_02_04_25_GOOD()
flat = FlatCorpus(flatAda, limit = 10)

print(flat[[5]])

for doc in flat:
    print(doc)
    #flatAda.save([{"tokens" : doc.words, "tags" : doc.tags}])

#print(adapter.load(1))
#print(adapter.load(3))
#print(adapter.load(3))
#adapter.save([{"a" : [1,2,3,7,8], "b" : [4]}, {"a" : [9,5, 8], "b" : [6]}])

[TaggedDocument(words=['svelte', 'baidumaps', 'set', 'baidu', 'map', 'component', 'svelte'], tags=['github:vulcangz/svelte-baidumaps', 'svelte-baidumaps', 'Svelte', 'baidu-map', 'baidumap', 'map', 'svelte'])]
TaggedDocument<['python', 'djvulibre', 'python', 'interface', 'djvulibre'], ['github:FriedrichFroebel/python-djvulibre', 'python-djvulibre', 'Cython', 'djvu', 'djvulibre']>
TaggedDocument<['hledger', 'robust', 'fast', 'intuitive', 'plain', 'text', 'accounting', 'tool', 'cli', 'tui', 'web', 'interface'], ['github:simonmichael/hledger', 'hledger', 'Haskell', 'accounting', 'bounties', 'bounty', 'command-line-tool', 'console-application', 'finance', 'haskell', 'haskell-applications', 'hledger', 'ledger', 'plaintext', 'plaintext-accounting', 'plaintextaccounting', 'tui', 'web-application']>
TaggedDocument<['rule', 'nixpkgs', 'rule', 'importing', 'nixpkgs', 'package', 'bazel'], ['github:tweag/rules_nixpkgs', 'rules_nixpkgs', 'Starlark', 'bazel', 'bazel-rules', 'nix', 'nixpkgs']>
TaggedD

In [4]:
import logging
logging.basicConfig(
    #filename="/home/trukhinmaksim/src/logs/09-04-25_autotunning.log",
    format='%(asctime)s : %(levelname)s : %(message)s',
    level=logging.INFO
)

In [5]:
trainAda = createTrainSetAdapter_02_04_25_GOOD()
train = FlatCorpus(trainAda, limit = 10)

testAda = createTestSetAdapter_02_04_25_GOOD()
test = FlatCorpus(testAda, limit = 10)

VECTOR_SIZE = 4
EPOCHS_NUMBER = 2
WORD_MIN_COUNT = 1
WINDOW_SIZE = 7
NEGATIVE_SAMPLES_AMOUNT = 6
SUBSAMPLING_THRESHOLD = 1e-5
ALPHA_INIT = 0.05
ALPHA_FINAL = 0.00001
DM_DBOW_MODE = "DM" # "DBOW" "DM+DBOW"

# finetunning is done by twicking model parameters
model = Model(
    vector_size =  VECTOR_SIZE, 
    window =       WINDOW_SIZE, 
    min_count =    WORD_MIN_COUNT, 
    epochs =       EPOCHS_NUMBER, 
    dm_dbow_mode = DM_DBOW_MODE,
    negative =     NEGATIVE_SAMPLES_AMOUNT,
    sample =       SUBSAMPLING_THRESHOLD,
    alpha_init =   ALPHA_INIT,
    alpha_final =  ALPHA_FINAL
)

print(train[[0]][0])
model.trainCorpus = train
model.testCorpus = [TaggedDocument(words = train[[1]][0].words, tags = train[[1]][0].tags)]

#model.train()
model.evaluate()

2025-04-14 16:07:32,678 : INFO : Model lifecycle event {'params': 'Model<dm/m,d4,n6,w7,s1e-05,t3>', 'datetime': '2025-04-14T16:07:32.677967', 'gensim': '4.3.3', 'python': '3.11.11 (main, Jan 14 2025, 05:22:51) [GCC 12.2.0]', 'platform': 'Linux-6.13.8-200.fc41.x86_64-x86_64-with-glibc2.36', 'event': 'created'}
2025-04-14 16:07:32,680 : INFO : collecting all words and their counts
2025-04-14 16:07:32,681 : INFO : PROGRESS: at example #0, processed 0 words (0 words/s), 0 word types, 0 tags
2025-04-14 16:07:32,682 : INFO : collected 66 word types and 10 unique tags from a corpus of 10 examples and 86 words
2025-04-14 16:07:32,683 : INFO : Creating a fresh vocabulary
2025-04-14 16:07:32,684 : INFO : Model lifecycle event {'msg': 'effective_min_count=1 retains 66 unique words (100.00% of original 66, drops 0)', 'datetime': '2025-04-14T16:07:32.684503', 'gensim': '4.3.3', 'python': '3.11.11 (main, Jan 14 2025, 05:22:51) [GCC 12.2.0]', 'platform': 'Linux-6.13.8-200.fc41.x86_64-x86_64-with-glib

TaggedDocument<['python', 'djvulibre', 'python', 'interface', 'djvulibre'], ['github:FriedrichFroebel/python-djvulibre', 'python-djvulibre', 'Cython', 'djvu', 'djvulibre']>


0.2

In [6]:
v = model.dv[0]

In [7]:
model.selectKmostSimilar(v, 10)

[(3, -0.017736465),
 (4, -0.010454547),
 (6, 0.061060876),
 (7, 0.13917464),
 (1, 0.18533166),
 (2, 0.29214755),
 (5, 0.69720995),
 (8, 0.6990359),
 (9, 0.74824166),
 (0, 0.9999999)]

In [8]:
train.onlyID = False
model.dv.most_similar([v], topn = 10)

[('github:FriedrichFroebel/python-djvulibre', 0.9999999403953552),
 ('github:janeczku/calibre-web', 0.748241662979126),
 ('github:Excision-Mail/Excision-Mail', 0.6990360021591187),
 ('github:vulcangz/svelte-baidumaps', 0.6972100734710693),
 ('github:tweag/rules_nixpkgs', 0.29214754700660706),
 ('github:simonmichael/hledger', 0.18533167243003845),
 ('github:spaskalev/karpsichord', 0.13917464017868042),
 ('github:matejak/argbash', 0.06106089800596237),
 ('github:jwilk/python-djvulibre', -0.010454473085701466),
 ('github:dspinellis/unix-history-make', -0.01773645728826523)]

In [9]:
topK = sorted(model.selectKmostSimilar(v, 10), key = lambda pair: pair[0])
topK

[(0, 0.9999999),
 (1, 0.18533166),
 (2, 0.29214755),
 (3, -0.017736465),
 (4, -0.010454547),
 (5, 0.69720995),
 (6, 0.061060876),
 (7, 0.13917464),
 (8, 0.6990359),
 (9, 0.74824166)]

In [10]:

trueRelevant = model.checkRelevants([p[0] for p in topK], ['github:FriedrichFroebel/python-djvulibre', 'python-djvulibre', 'Cython', 'djvu', 'djvulibre'])
trueRelevant

array([1., 0., 0., 0., 1., 0., 0., 0., 0., 0.])

In [11]:
train.onlyID = False
for doc in train:
    print(doc.tags)

['github:FriedrichFroebel/python-djvulibre', 'python-djvulibre', 'Cython', 'djvu', 'djvulibre']
['github:simonmichael/hledger', 'hledger', 'Haskell', 'accounting', 'bounties', 'bounty', 'command-line-tool', 'console-application', 'finance', 'haskell', 'haskell-applications', 'hledger', 'ledger', 'plaintext', 'plaintext-accounting', 'plaintextaccounting', 'tui', 'web-application']
['github:tweag/rules_nixpkgs', 'rules_nixpkgs', 'Starlark', 'bazel', 'bazel-rules', 'nix', 'nixpkgs']
['github:dspinellis/unix-history-make', 'unix-history-make', 'Roff', 'bell-labs', 'evolution', 'history', 'unix']
['github:jwilk/python-djvulibre', 'python-djvulibre', 'Cython', 'djvu']
['github:vulcangz/svelte-baidumaps', 'svelte-baidumaps', 'Svelte', 'baidu-map', 'baidumap', 'map', 'svelte']
['github:matejak/argbash', 'argbash', 'M4', 'argument-parsing', 'code-generator', 'shell-scripts']
['github:spaskalev/karpsichord', 'karpsichord', 'Faust', 'baroque', 'harpsichord', 'instrument', 'music', 'musical-instru